In [2]:
import MDAnalysis as mda
import numpy as np
import torch
from scipy.special import erf
import matplotlib.pyplot as plt

## This is for checking vanilla Chill+

In [3]:
u = mda.Universe('bias.gro', 'bias_short.xtc')

In [22]:
pos = u.atoms.positions
box = u.dimensions[:3]

In [ ]:
import numpy as np

def pbc_distance(atom1_pos: np.ndarray, atom2_pos: np.ndarray, box: np.ndarray) -> np.ndarray:
    """
    Calculate the PBC-corrected pairwise distances between two sets of atoms.
    
    Parameters:
    - atom1_pos: (n1,3) array representing the positions of n1 atoms.
    - atom2_pos: (n2,3) array representing the positions of n2 atoms.
    - box: (3,) array representing the box dimensions in x, y, and z directions.
    
    Returns:
    - A (n1,n2) array where each element (i,j) represents the PBC-corrected distance between atom i from atom1_pos and atom j from atom2_pos.
    """
    
    rik = atom1_pos[:, np.newaxis,:] - atom2_pos[np.newaxis, :,:]
    rik = rik - np.round(rik / box) * box
    rsphki = np.sqrt( (rik**2).sum(axis = -1) )
    
    return rsphki
    
    

In [26]:
dist

array([[66.11066 , 66.14985 , 66.065155, ..., 32.31616 , 32.330635,
        32.743   ]], dtype=float32)

In [28]:
pos[0]

array([ 7.9300003,  2.87     , 30.300003 ], dtype=float32)

In [30]:
dist[0,0]

66.11066

In [ ]:
def identifyChillPlus(u:mda.Universe, harmonic_index=3, eclipsed_range=[-0.45, 0.18], staggered_range=[-1.0, -0.8]):
    return 0

In [1]:
def identifyChillPlus(u:mda.Universe, degree=3):
    """
    Args:
    Returns:
    """
    # number of m calculated from degree
    num_m = degree * 2 + 1
    # select the atoms with name OW
    pos = u.select_atoms("name OW").atoms.positions/10
    box = u.dimensions[:3]/10
    # find the nearest 4 neighbors
    dist= pbcDistance(pos[:,np.newaxis,:] - pos, box)
    dist= np.sqrt((dist**2).sum(axis=-1))
    argsorted = np.argsort(dist, axis=1)[:,1:5]
    # define the qlm
    qlm = np.zeros((argsorted.shape[0],), dtype=complex)
    for (i,a) in enumerate(argsorted):
        qlmi = np.zeros((num_m, ), dtype=complex)
        for ind in a:
            r = pbcDistance(pos[ind] - pos[i], box)
            r = r / np.sqrt((r**2).sum())
            if r[0] >0 and r[1]>0:
                azimuth = np.arctan(r[0]/r[1]) 
            elif r[0] <0 and r[1]>0:
                azimuth = np.arctan(r[0]/r[1]) + 2*np.pi
            else:
                azimuth = np.arctan(r[0]/r[1]) + np.pi
        
            zenith  = np.arccos(r[2])
            l = np.arange(-3,4,1)
            res = sp.sph_harm(l, 3, azimuth, zenith)
            qlmi = qlmi + res
        qlm[i] = qlmi/4
    cij = (qlm[:,np.newaxis,:] * np.conjugate(qlm[np.newaxis,:,:])).sum(axis=-1)
    qi  = np.sqrt((qlm * np.conjugate(qlm)).sum(axis=-1,keepdims=True))
    qij = qi[:,np.newaxis,:] * qi
    qij = qij.reshape(1024,1024)
    cij = cij/qij

NameError: name 'mda' is not defined